In [1]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
import time
import pandas as pd
import gurobipy as grb
import datetime
import random
np.set_printoptions(edgeitems=15,linewidth=600)

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.optimization.mip_model import *

In [2]:
start = time.time()

In [3]:
data = pd.read_excel('../data/data_template_blank.xlsx').head(20)
data['current_age'] = datetime.datetime.now().year - pd.to_datetime(data.purchasedate).dt.year
data = data.reset_index().rename({"index":"vehicle_idx"},axis=1)
data['county'] = 'Baltimore County'

In [4]:
mip = MIP(data,UI_params)

In [5]:
keepSchedules = mip.make_keep_schedules()

In [6]:
age = mip.get_vehicle_age(keepSchedules)#[50:80]

In [31]:
annual_mileage = mip.make_mileage()

In [8]:
odometer = mip.make_odometer(annual_mileage,age)

In [9]:
acquisition = mip.get_acquisition_cost()

In [10]:
is_ice,is_ev = mip.get_vehicle_type_trackers()

In [11]:
consumables = mip.get_consumables(annual_mileage,is_ice,is_ev)

In [32]:
emissions = mip.get_emissions(annual_mileage,is_ice,is_ev)

In [13]:
maintenance = mip.get_maintenance_cost(odometer)

In [14]:
infeasible_filter = mip.find_infeasible_schedules(odometer,age)
# mip.replacement_schedules[~infeasible_filter]

In [15]:
m,x,vehicles,validSchedulesPerVehicle = mip.make_model(consumables,acquisition,maintenance,emissions,infeasible_filter)

Using license file C:\Users\elynch\gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolSolutions to 3
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter TimeLimit to 30.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 53 rows, 1888 columns and 36085 nonzeros
Model fingerprint: 0x7aa1dc46
Variable types: 2 continuous, 1886 integer (1886 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [2e+01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 43738.800000
Presolve removed 33 rows and 0 columns
Presolve time: 0.01s
Presolved: 20 rows, 1888 columns, 1886 nonzeros
Variable types: 2 continuous, 1886 integer (1886 binary)
Found heuristic solution: objective 30432.500000

Root

In [16]:
end = time.time()
print("--- %s seconds ---" % (time.time() - start))


--- 5.108161926269531 seconds ---


In [33]:
#multiple solutions
options = {}

# options = ['A','B','C']
for sol in range(0,mip.numDesiredSolutions):
    schedules = []
    acquisition_costs = []
    mx_costs = []
    consumables_costs = []
    emissions_amts = []
    conversions = []
    print()
    print(f'Option: {sol}')
    m.setParam('SolutionNumber',sol)
#     print(f'obj:{m.getObjective().getValue()}')
    
    for v in vehicles:
        for s in validSchedulesPerVehicle[v]:
            if x[v,s].xn==1:
#                 print(f'   Vehicle: {v+1} Schedule: {s} {mip.replacement_schedules[v,s]}')  
                schedules.append([v]+[s]+list(mip.replacement_schedules[v,s]))
                acquisition_costs.append([v]+[s]+list(acquisition[v,s]))
                mx_costs.append([v]+[s]+list(maintenance[v,s]))
                consumables_costs.append([v]+[s]+list(consumables[v,s]))
                emissions_amts.append([v]+[s]+list(emissions[v,s]))
                conversions.append([v]+[s]+list(is_ev[v,s]))
    options[sol,'schedules'] = schedules
    options[sol,'acquisition_costs'] = acquisition_costs
    options[sol,'mx_costs'] = mx_costs
    options[sol,'consumables_costs'] = consumables_costs
    options[sol,'emissions_amts'] = emissions_amts
    options[sol,'conversions'] = conversions


Option: 0

Option: 1

Option: 2


In [18]:
option1 = pd.DataFrame(options[0,'schedules'],columns=['vehicle_idx','solution_idx']+mip.years)

In [19]:
data = data.merge(option1,on='vehicle_idx',how='left')

In [20]:
data.groupby(['county'])[mip.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
county,,,,,,,,,,,,,,,,
Baltimore County,15,3,,1,,,1,,,1,,1,,,,


In [21]:
data.groupby(['dept_name'])[mip.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
dept_name,,,,,,,,,,,,,,,,
Baltimore County Department of Library,1,,,,,,,,,,,,,,,
Bureau of Corrections,1,,,,,,,,,,,,,,,
DPW: Bureau of Utilties (Pumping/Treatment Plants),2,,,,,,,,,,,,,,,
DPW: Equipment Maintenance,1,,,,,,,,,,,,,,,
DPW: Solid Waste Management (Refuse Disposal),1,,,,,,,,,,,,,,,
DPW: Traffic Signal Operations,1,,,,,,,,,,,,,,,
Department of Health (Bioterrorism Program),1,,,,,,,,,,,,,,,
OIT: Electronic Services/Telecommunications,1,1,,,,,,,,,,,,,,
Police Department - Administrative/Technical Services,2,,,,,,,,,,,,,,,


In [22]:
data.groupby(['dept_name','vehicledescription'])[mip.years].sum().replace({0:''})

2022  \
dept_name                                          vehicledescription                     
Baltimore County Department of Library             2002 FORD VAN CARGO'               1   
Bureau of Corrections                              2001 GMC VAN 12PSG'                1   
DPW: Bureau of Utilties (Pumping/Treatment Plants) 2001 CHEVROLET 1.0 TON UT'         1   
                                                   2003 CHEVROLET C4500 UTIL BODY'    1   
DPW: Equipment Maintenance                         2003 CHEVROLET C4500 UTIL BODY'    1   
DPW: Solid Waste Management (Refuse Disposal)      1999 GMC 3/4 TON PU'               1   
DPW: Traffic Signal Operations                     2000 GMC 3/4 TON PU'               1   
Department of Health (Bioterrorism Program)        2000 GMC 3/4 TON PU'               1   
OIT: Electronic Services/Telecommunications        2000 CHEVROLET VAN CARGO'          1   
                                                   2000 GMC 1.0 TON UT'                   
Police Department - Administrative/Technical Se... 2002 FORD TAURUS'                  1   
                                                   2003 CHEVROLET Astro'              1   
Police Department - Support Operations             2003 Harley Davidson FLHT'         2   
Property Management - Building Maintenance         2002 FORD VAN CARGO'               1   
Recreation and Parks (Recreation Services)         2001 GMC VAN CARGO'                1   

                                                                                   2023  \
dept_name                                          vehicledescription                     
Baltimore County Department of Library             2002 FORD VAN CARGO'                   
Bureau of Corrections                              2001 GMC VAN 12PSG'                    
DPW: Bureau of Utilties (Pumping/Treatment Plants) 2001 CHEVROLET 1.0 TON UT'             
                                                   2003 CHEVROLET C4500 UTIL BODY'        
DPW: Equipment Maintenance                         2003 CHEVROLET C4500 UTIL BODY'        
DPW: Solid Waste Management (Refuse Disposal)      1999 GMC 3/4 TON PU'                   
DPW: Traffic Signal Operations                     2000 GMC 3/4 TON PU'                   
Department of Health (Bioterrorism Program)        2000 GMC 3/4 TON PU'                   
OIT: Electronic Services/Telecommunications        2000 CHEVROLET VAN CARGO'              
                                                   2000 GMC 1.0 TON UT'               1   
Police Department - Administrative/Technical Se... 2002 FORD TAURUS'                      
                                                   2003 CHEVROLET Astro'                  
Police Department - Support Operations             2003 Harley Davidson FLHT'         2   
Property Management - Building Maintenance         2002 FORD VAN CARGO'                   
Recreation and Parks (Recreation Services)         2001 GMC VAN CARGO'                    

                                                                                   2024  \
dept_name                                          vehicledescription                     
Baltimore County Department of Library             2002 FORD VAN CARGO'                   
Bureau of Corrections                              2001 GMC VAN 12PSG'                    
DPW: Bureau of Utilties (Pumping/Treatment Plants) 2001 CHEVROLET 1.0 TON UT'             
                                                   2003 CHEVROLET C4500 UTIL BODY'        
DPW: Equipment Maintenance                         2003 CHEVROLET C4500 UTIL BODY'        
DPW: Solid Waste Management (Refuse Disposal)      1999 GMC 3/4 TON PU'                   
DPW: Traffic Signal Operations                     2000 GMC 3/4 TON PU'                   
Department of Health (Bioterrorism Program)        2000 GMC 3/4 TON PU'                   
OIT: Electronic Services/Telecommunications        2000 CHEVROLET VAN CARGO' 

In [23]:
data.groupby(['equipmentid'])[mip.years].sum().replace({0:''})

,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
equipmentid,,,,,,,,,,,,,,,,
24399,1,,,,,,,,,,,,,,,
26142,1,,,,,,,,,,,,,,,
26145,1,,,,,,,,,,,,,,,
26521,1,,,,,,,,,,,,,,,
26526,1,,,,,,,,,,,,,,,
26563,1,,,,,,,,,,,,,,,
26566,,1,,,,,,,,,,,,,,
26783,1,,,,,,,,,,,,,,,
26785,1,,,,,,,,,,,,,,,


In [37]:
print('   acquisition_costs')
pd.DataFrame(pd.DataFrame(options[0,'acquisition_costs'],columns=['vehicle_idx','solution_idx']+mip.years)[mip.years].sum()).T.astype(int)

   acquisition_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,803000,80000,0,20500,0,0,20500,0,0,20500,0,20500,0,0,0,0


In [38]:
print('   consumables_costs')
pd.DataFrame(pd.DataFrame(options[0,'consumables_costs'],columns=['vehicle_idx','solution_idx']+mip.years)[mip.years].sum()).T.astype(int)

   consumables_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,1711,3285,2402,2748,2377,2677,2605,3065,2630,2832,2815,2688,2810,2722,2707,2908


In [39]:
print('   mx_costs')
pd.DataFrame(pd.DataFrame(options[0,'mx_costs'],columns=['vehicle_idx','solution_idx']+mip.years)[mip.years].sum()).T.astype(int)

   mx_costs


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,20000,20000,20600,21600,22200,22400,22400,22600,22800,22800,23000,23000,23000,22800,23200,23400


In [40]:
print('   emissions_amts')
pd.DataFrame(pd.DataFrame(options[0,'emissions_amts'],columns=['vehicle_idx','solution_idx']+mip.years)[mip.years].sum()).T.astype(int)

   emissions_amts


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,905,377,393,121,121,17,0,0,0,0,0,0,0,0,0,0


In [41]:
print('   EV/Hybrid Inventory')
pd.DataFrame(pd.DataFrame(options[0,'conversions'],columns=['vehicle_idx','solution_idx']+mip.years)[mip.years].sum()).T.astype(int)

   EV/Hybrid Inventory


,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,15,18,18,19,19,19,20,20,20,20,20,20,20,20,20,20


In [29]:
option1

,vehicle_idx,solution_idx,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
0,0,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6,14,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,8,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
v = 1
s = 11
print(f'schedule {s}: {mip.replacement_schedules[0,s,:]}')
print()
print(f'age: {age[v,s]}')
print()
print(f'annual_mileage: {annual_mileage[v,s]}')
print()
print(f'odometer: {odometer[v,s]}')
print()
print(f'acquisition cost: {acquisition[v,s]}')
print()
print(f'maintenance cost: {maintenance[v,s]}')
print()
print(f'consumables cost: {consumables[v,s]}')
print()
print(f'emissions: {emissions[v,s]}')

schedule 11: [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]

age: [20 21 22 23  0  1  2  3  4  5  6  7  8  9 10 11]

annual_mileage: [3411. 5491. 2850. 4096. 3171. 3044. 3632. 4438. 4058. 4104. 3121. 2783. 4198. 4928. 4324. 3193.]

odometer: [ 68220. 115311.  62700.  94208.      0.   3044.   7264.  13314.  16232.  20520.  18726.  19481.  33584.  44352.  43240.  35123.]

acquisition cost: [    0.     0.     0.     0. 46000.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]

maintenance cost: [1200. 1200. 1200. 1200. 1000. 1000. 1000. 1200. 1200. 1200. 1200. 1200. 1200. 1200. 1200. 1200.]

consumables cost: [401. 645. 335. 481. 113. 108. 129. 158. 144. 146. 111.  99. 149. 175. 154. 114.]

emissions: [413. 665. 345. 496.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
